In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sympy as sp
from Modules.classes import Operator, Coefficient, Term, Expression

In [8]:
X = Operator('sigma_x', mat_representation=sp.Matrix([[0, 1], [1, 0]]), subspace='spin')
Y = Operator('tau_y', mat_representation=sp.Matrix([[0, -sp.I], [sp.I, 0]]), subspace='spin')
a = Operator('a', subspace='oscillator', is_infinite=True, mat_representation=sp.sqrt(sp.Rational(1, 2)) * sp.Matrix([[0, 1], [0, 0]]))

In [9]:
(X * Y * a).is_infinite

True

In [ ]:
g = Coefficient('g', order=1)
h = Coefficient('h', order=2)

res = g * h
res * X + g*Y

g*h sigma_x + g tau_y

1 sigma_x*tau_y**5 + -1 tau_y*sigma_x*tau_y**4 + -1 tau_y*sigma_x*tau_y**4 + 1 tau_y**2*sigma_x*tau_y**3 + -1 tau_y*sigma_x*tau_y**4 + 1 tau_y**2*sigma_x*tau_y**3 + 1 tau_y**2*sigma_x*tau_y**3 + -1 tau_y**3*sigma_x*tau_y**2 + -1 tau_y*sigma_x*tau_y**4 + 1 tau_y**2*sigma_x*tau_y**3 + 1 tau_y**2*sigma_x*tau_y**3 + -1 tau_y**3*sigma_x*tau_y**2 + 1 tau_y**2*sigma_x*tau_y**3 + -1 tau_y**3*sigma_x*tau_y**2 + -1 tau_y**3*sigma_x*tau_y**2 + 1 tau_y**4*sigma_x*tau_y + -1 tau_y*sigma_x*tau_y**4 + 1 tau_y**2*sigma_x*tau_y**3 + 1 tau_y**2*sigma_x*tau_y**3 + -1 tau_y**3*sigma_x*tau_y**2 + 1 tau_y**2*sigma_x*tau_y**3 + -1 tau_y**3*sigma_x*tau_y**2 + -1 tau_y**3*sigma_x*tau_y**2 + 1 tau_y**4*sigma_x*tau_y + 1 tau_y**2*sigma_x*tau_y**3 + -1 tau_y**3*sigma_x*tau_y**2 + -1 tau_y**3*sigma_x*tau_y**2 + 1 tau_y**4*sigma_x*tau_y + -1 tau_y**3*sigma_x*tau_y**2 + 1 tau_y**4*sigma_x*tau_y + 1 tau_y**4*sigma_x*tau_y + -1 tau_y**5*sigma_x